<a target="_blank" href="https://colab.research.google.com/github/ZHAW-ZAV/TSO-FS25/blob/main/01_python_basics/first_steps_pandas.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import sys
import os

IN_COLAB = "google.colab" in sys.modules

file_id_co2 = "1XsuqMub8M3SdP50X1MIwAci-lsYS0X9t"

if IN_COLAB:
    path_to_co2 = "/content/data/co2_emmissions_by_state_2023.csv"
    os.makedirs(os.path.dirname(path_to_co2), exist_ok=True)
    !gdown "https://drive.google.com/uc?id={file_id_co2}" -O "{path_to_co2}"
else:
    import gdown

    url = f"https://drive.google.com/uc?id={file_id_co2}"
    path_to_co2 = "data/co2_emmissions_by_state_2023.csv"
    os.makedirs(os.path.dirname(path_to_co2), exist_ok=True)
    gdown.download(url, path_to_co2, quiet=False)


The code above it to load the data, don't modify.

***Notebook starts here***

# Introduction to Pandas


## Importing Pandas

In [5]:
import pandas as pd

## Reading Data

In [ ]:
# You can read all kinds of files with pandas, not just csv,
# but also excel, json, etc. with pd.read_excel, pd.read_json
co2 = pd.read_csv(path_to_co2)

## Exploring Data

In [ ]:
print(f"The DataFrame has {len(co2)} rows.")
print(f"The columns in the DataFrame are: {co2.columns}")

In [ ]:
# Let's take a look at the first few rows
co2.head()

In [ ]:
# Let's take a look at the last few rows
co2.tail()

In [ ]:
# data types of the columns
co2.info()

In [ ]:
# Some descriptive statistics
co2.describe()

In [ ]:
# Let's see the unique values in the "STATE_NAME" column
co2["STATE_NAME"].unique()

In [ ]:
# How many unique values are there in the "STATE_NAME" column?
co2["STATE_NAME"].value_counts()

## Filtering and Slicing Data

In [ ]:
# classical approach
co2_ch = co2[co2["STATE_NAME"] == "SWITZERLAND"]
co2_ch

In [ ]:
# using query()
co2_country = co2.query("STATE_NAME == 'SWITZERLAND'")
co2_country

In [ ]:
# with query, you can use variables and complex expressions
country = "ESTONIA"
co2_country = co2.query("STATE_NAME == @country and MONTH <= 6")
co2_country

In [ ]:
# show only the columns we are interested in
co2[["STATE_NAME", "MONTH", "CO2_QTY_TONNES"]]

## Aggregating and Sorting Data

In [ ]:
print(f"Total CO2 emissions: {co2['CO2_QTY_TONNES'].sum()}")
print(f"Mean CO2 emissions: {co2['CO2_QTY_TONNES'].mean()}")

# let's display this neatly
print(f"Total CO2 emissions: {co2['CO2_QTY_TONNES'].sum():.0f}")
print(f"Mean CO2 emissions: {co2['CO2_QTY_TONNES'].mean():.1f}")


### Grouping data, single aggregation

In [ ]:
# group by country and sum the CO2 emissions

# this is a bit more advanced, let's break it down
# 1. group by country: co2.groupby("STATE_NAME")
# 2. select the column we are interested in: ["CO2_QTY_TONNES"]
# 3. sum the values: .sum()

co2_country = co2.groupby("STATE_NAME")["CO2_QTY_TONNES"].sum()
co2_country

In [ ]:
# same as above, but sorted
#
# Hint: you can split the code into multiple lines to make it more readable,
#       just wrap the code in parentheses

co2_country = (
    co2.groupby("STATE_NAME")["CO2_QTY_TONNES"].sum().sort_values(ascending=False)
)
co2_country

### Grouping data, multiple aggregations

In [ ]:
# you can also use the .agg() method to apply multiple functions at once
# for example, we want the sum of the total traffic (TF) and the sum and mean of the CO2 emissions
#
# 1. group by country: co2.groupby("STATE_NAME")
# 2. use the .agg() method to apply multiple functions at once
#    it takes a dictionary as an argument, where the keys are the column names
#    and the values are the functions to apply
# 3. sort the values by the sum of the CO2 emissions
co2_country = (
    co2.groupby("STATE_NAME")
    .agg({"TF": "sum", "CO2_QTY_TONNES": ["sum", "mean"]})
    .sort_values(by=("CO2_QTY_TONNES", "sum"), ascending=False)
)
co2_country

## Modifying Data

In [ ]:
# let's say we want to add a column with the CO2 emissions per movement
# we can do this by dividing the CO2 emissions by the total traffic
co2["CO2_PER_TF"] = co2["CO2_QTY_TONNES"] / co2["TF"]
co2.sort_values("CO2_PER_TF", ascending=False)